Extract from:

Data Sources:
- [NF Source](https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix)
- [HL Source](https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix)
- [HBO Source](https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix)

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

Data Sources for OC

In [0]:
NF = 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix'
HBO = ''
HL = 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Hulu'

Tools: 
- Get_Rotten: Extract Given URL

In [0]:
def get_rotten(rotten_loc):
  finder = re.compile('[0-9]{1,3}')
  missing = re.compile('Tomatometer Not Available...')

  # Scrape
  rotten_requested = requests.get(rotten_loc)
  rotten_extract = rotten_requested.text
  rotten_soup = BeautifulSoup(rotten_extract,'html5lib')
  
  # Check 404
  if rotten_soup.find('h1').get_text().strip() == '404 - Not Found':
    return {'url':rotten_loc, 'critic':'-', 'audience':'-','status':'404'}
 
  else:
    # Critic
    try:
      crit = rotten_soup.find('div',{'id':'all-critics-numbers'})
      # Check if Missing
      if missing.search(crit.get_text()):
        crit = '-'
      # Extract critic if not missing
      else:
        crit = finder.search(crit.get_text()).group(0)
    # if fail
    except:
      crit = '-'
    
    # Extract Audience
    try: 
      aud = rotten_soup.find('div',{'class':'audience-score meter'})
      aud = finder.search(aud.get_text()).group(0)
    # If Fail
    except:
      aud = '-'
                            
    return {'url':rotten_loc, 'critic':crit, 'audience':aud,'status':'success'}
  
  
def get_wiki(wiki_loc):
  wiki_requested = requests.get(wiki_loc)
  wiki_extract = wiki_requested.text
  wiki_soup = BeautifulSoup(wiki_extract,'html5lib')
  wiki_tables = wiki_soup.findChildren('table')
  wiki_row_collected = []

  for table in wiki_tables:
    wiki_rows = table.find_all('tr')
    for row in wiki_rows:
      row = row.find_all('a')
      row = [i.get('href') for i in row]
      wiki_row_collected += [row]
  wiki_row_collected = [i for i in wiki_row_collected if len(i) != 0]
  
  return wiki_row_collected

In [0]:
test_1 = get_wiki(wiki_loc='https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Hulu')[0:10]
test_2 = get_wiki(wiki_loc='https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix')[0:10]
test_3 = get_wiki(wiki_loc='https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_HBO')[0:10]
test_4 = get_wiki(wiki_loc = 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Amazon')[0:10]